In [22]:
import pandas as pd
import datetime

#api
import praw

#pre-processing
import nltk 
import re
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
#from nltk.corpus import stopwords
# nltk.download()

#analysis
from sklearn.decomposition import TruncatedSVD

In [4]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


Get top posts of all time

In [5]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

                                                 title  score      id  \
0    Elon Musk reminds me of a rich billionaire who...  39137  lce6zm   
1    CryptoNick is deleting all of his BitConnect v...  26495  7r0ftz   
2    Reminder: Robinhood blocked several stocks fro...  26196  lfrslo   
3    I will tell you exactly what is going on here,...  20151  7vga1y   
4    Robinhood is launching a Crypto Trading app to...  19960  7sx5ze   
..                                                 ...    ...     ...   
926                              Remember to zoom out!   1480  7lhoxn   
927  Ether Trades Above $500 for the First Time Sin...   1481  jxm1kb   
928  I found this post from 9 years ago on reddit, ...   1475  kh8ptc   
929                                  [Trigger Warning]   1481  fp92hd   
930  Perfect Explanation Of CryptoCurrency - He Nai...   1472  c2u7ob   

          subreddit                                                url  \
0    CryptoCurrency  https://www.reddit.com/r/Cry

Get top coments from each top post

In [6]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        for x in comments[:20]:
            top_comments.append([x,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

                                                 comment                date
0      If your life savings hinges on a Tweet from a ... 2021-02-04 12:02:16
1      I seriously don’t understand the obsession wit... 2021-02-04 12:02:16
2      Tbh u can't place all the blame on Elon tho, t... 2021-02-04 12:02:16
3      seriously, there's so much more relevant peopl... 2021-02-04 12:02:16
4      If I'm Elon, I put hundreds of millions into D... 2021-02-04 12:02:16
...                                                  ...                 ...
18596      I hate dough bags, but this one was funny AF. 2019-06-20 10:09:41
18597  It rubs the blockchain on its skin whenever it... 2019-06-20 10:09:41
18598  Perfect explanation of crypto currency? Not re... 2019-06-20 10:09:41
18599          "it's a series of tubes on the interwebs" 2019-06-20 10:09:41
18600                                     she's a puppet 2019-06-20 10:09:41

[18601 rows x 2 columns]


Tokenization

In [21]:
comments = list(top_comments['comment'])

comments_cleaned = [re.sub('\w*\d\w*', '', i) for i in comments]

# set(stopwords.words('english'))
cv = CountVectorizer(stop_words='english')
X= cv.fit_transform(comments_cleaned)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

,__,___,____,_______,_________,__________,_______________________________,__coin,_a,_account,...,zuckerberg,zuckerbucks,zuckercash,zunes,åtæreøm,în,ðoge,škoda,ебитес,ಠ_ಠ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


analysis

In [23]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(X)
lsa.explained_variance_ratio_

array([0.0590808 , 0.03356333, 0.01808756, 0.01350056, 0.0118891 ])

In [27]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2", "component_3","component_4", "component_5"],
             columns = cv.get_feature_names())
topic_word

,__,___,____,_______,_________,__________,_______________________________,__coin,_a,_account,...,zuckerberg,zuckerbucks,zuckercash,zunes,åtæreøm,în,ðoge,škoda,ебитес,ಠ_ಠ
component_1,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0
component_2,0.000,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000,0.000,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0
component_3,0.001,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000,-0.000,...,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
component_4,-0.000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.002,-0.001,...,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0
component_5,-0.000,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.000,0.000,...,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0


In [28]:
#x